# New developments in deep learning - R-GCNs
Isotropic Graph Neural Networks 
- Representations are learned via differentiable message passing scheme 
-  All neighbors are treated as equally important 
-  Starting points: 
  -  Kipf & Welling: “Semi-Supervised Classification with Graph Convolutional Networks” (https://arxiv.org/abs/1609.02907) 
  -  Schlichtkrull et al.: “Modeling Relational Data with Graph Convolutional Networks” (https://arxiv.org/abs/1703.06103) 
- Task: 
  - Implement Relational Graph convolutional Neural Network for Node Classification

Blog posts:
- https://towardsdatascience.com/how-to-do-deep-learning-on-graphs-with-graph-convolutional-networks-7d2250723780
- http://tkipf.github.io/graph-convolutional-networks/

Jupyter notebook tutorial:
- https://github.com/TobiasSkovgaardJepsen/posts/blob/master/HowToDoDeepLearningOnGraphsWithGraphConvolutionalNetworks/Part2_SemiSupervisedLearningWithSpectralGraphConvolutions/notebook.ipynb

Keras implementation:
- https://github.com/tkipf/relational-gcn
PyTorch implementations
- https://github.com/tkipf/pygcn 
- https://github.com/masakicktashiro/rgcn_pytorch_implementation
- https://github.com/mjDelta/relation-gcn-pytorch

Dataset:
- https://ogb.stanford.edu
- https://ogb.stanford.edu/docs/nodeprop/#ogbn-proteins
- https://ogb.stanford.edu/docs/leader_nodeprop/#ogbn-proteins 

## Relational GCNs
Extension of GCNs: Use a set of relation-specific weight matrices $W_r^{(l)}$, where $r \in R$ denotes the relation type

Propagation model:
> $h_i^{l+1} = \sigma\left(\sum_{r\in R}\sum_{j\in N^r_i}\frac{1}{c_{i,r}}W_r^{(l)}h_j^{(l)}+\underbrace{W_0^{(l)}h_i^{(l)}}_{\text{self-connection}}\right)$

where 
- $N^r_i$ denotes the set of neighbor indices of node $i$ under relation $r \in R$, 
- $c_{i,r}$ is a problem-specific normalization constant that can either be learned or chosen in advance (such as $c_{i,r} = |N_i^r|$).

Neural network layer update: evaluate message passing update in parallel for every node $i \in V$.

Parameter sharing for highly- multi-relational data: basis decomposition of relation-specific weight matrices
> $W_r^{(l)} = \sum^B_{b=1}a^{(l)}_{r,b}V_b^{(l)}$

Linear combination of basis transformations $V_b^{(l)} \in \mathbb{R}^{d^{(l+1)}\times d^{(l)}}$ with learnable coefficients $a^{(l)}_{r,b}$ such that only the coefficients depend on $r$. $B$, the number of basis functions, is a hyperparameter.

For entity classification as described in the paper minimize:
> $L = -\sum_{i\in Y}\sum^K_{k=1}t_{i,k}\ln h_{i,k}^{(l)}$

whre:
- $Y$ is the set of node indices with labels
- $K$ is the number of classes (?)
- $t_{i,k}$ is the ground-truth label
- $h_{i,k}^{(l)}$ is the $k$-th entry of network ouput for $i$-th labeled node

# Training and evaluation
- 2 layer model with 16 hidden units (dimension of hidden node representation)
- 50 epochs with learning rate 0.01 using Adam optimizer
- normalization constant $c_{i,r} = |N_i^r|$, i.e. average all incoming messages from a particular relation type
- $l2$ penalty on first layer weights $C_{l2} \in \{0, 5\cdot 10^{-4}\}$
- number of basis functions $B \in \{0, 10, 20, 30, 40\}$

Results reported
- Accuracy and standard error over 10 runs

## Implementation

### Installing requirements

### Loading the data

In [ ]:
d_name = 'ogbn-proteins'

In [ ]:
from ogb.nodeproppred import NodePropPredDataset

dataset = NodePropPredDataset(name = d_name)

split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]
graph, label = dataset[0] # graph: library-agnostic graph object

The library-agnostic graph object is a dictionary containing the following keys: `edge_index`, `edge_feat`, `node_feat`, and `num_nodes`, which are detailed below.

- `edge_index`: numpy arrays of shape (2, `num_edges`), where each column represents an edge. The first row and the second row represent the indices of source and target nodes. Undirected edges are represented by bi-directional edges.
- `edge_feat`: numpy arrays of shape (`num_edges`, `edgefeat_dim`), where `edgefeat_dim` is the dimensionality of edge features and i-th row represents the feature of i-th edge. This can be None if no input edge features are available.
- `node_feat`: numpy arrays of shape (`num_nodes`, `nodefeat_dim`), where `nodefeat_dim` is the dimensionality of node features and i-th row represents the feature of i-th node. This can be None if no input node features are available.
- `num_nodes`: number of nodes in the graph.

###  Evaluation

Expected input format of Evaluator for ogbn-proteins
`{'y_true': y_true, 'y_pred': y_pred}`
- `y_true`: numpy ndarray or torch tensor of shape (`num_node`, `num_task`)
- `y_pred`: numpy ndarray or torch tensor of shape (`num_node`, `num_task`)
where y_pred stores score values (for computing ROC-AUC),
num_task is 112, and each row corresponds to one node.

Expected output format of Evaluator for ogbn-proteins
`{'rocauc': rocauc}`
- `rocauc` (float): ROC-AUC score averaged across 112 task(s)

In [ ]:
from ogb.nodeproppred import Evaluator

evaluator = Evaluator(name = d_namat) 

In [ ]:
# In most cases, input_dict is
# input_dict = {"y_true": y_true, "y_pred": y_pred}
result_dict = evaluator.eval(input_dict)